A simple bank account starts with $1000. Two "customers" (threads) run concurrently:

Customer 1: Deposits $10, 50 times (+$500 total).
Customer 2: Withdraws $5, 50 times (-$250 total).

In [1]:
import threading
import time

In [4]:
class CASBankAccount:
    def __init__(self, initial_balance=1000):
        self.balance = initial_balance
        self._lock = threading.Lock()  # Simulates atomicity (real CAS: CPU instr)

    def cas_deposit(self, expected, amount):
        with self._lock:
            current = self.balance
            if current == expected:
                self.balance = current + amount
                return True, self.balance
            return False, current

    def deposit(self, amount):
        expected = self.balance
        while True:
            success, new_bal = self.cas_deposit(expected, amount)
            if success:
                return new_bal
            expected = new_bal
            time.sleep(0.000001)  # Tiny delay to force some retries

    def cas_withdraw(self, expected, amount):
        with self._lock:
            current = self.balance
            if current == expected and current >= amount:
                self.balance = current - amount
                return True, self.balance
            return False, current

    def withdraw(self, amount):
        expected = self.balance
        while True:
            success, new_bal = self.cas_withdraw(expected, amount)
            if success:
                return new_bal
            expected = new_bal
            time.sleep(0.000001)

def customer1(account):
    for _ in range(50):
        account.deposit(10)
    print("Customer 1 done")

def customer2(account):
    for _ in range(50):
        account.withdraw(5)
    print("Customer 2 done")

if __name__ == "__main__":
    account = CASBankAccount(1000)
    t1 = threading.Thread(target=customer1, args=(account,))
    t2 = threading.Thread(target=customer2, args=(account,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    print(f"Final balance: {account.balance} (Expected: 1000 + 500 - 250 = 1250)")

Customer 1 done
Customer 2 done
Final balance: 1250 (Expected: 1000 + 500 - 250 = 1250)


I DIDNT understand ABA..! ;)